<a href="https://colab.research.google.com/github/krishna-kenny/nbaWinNeuralNetModel/blob/main/nba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nba_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 3.9 MB/s eta 0:00:00


In [2]:
import time
import pandas as pd
from nba_api.stats.endpoints import TeamInfoCommon, TeamGameLogs, PlayerGameLogs, LeagueGameFinder, LeagueLeaders, PlayerCareerStats
from nba_api.stats.static import teams

# Maximum number of retries for each API call
MAX_RETRIES = 3
# Define the list of seasons
seasons = ["2024-25"]

In [3]:
def fetch_with_retries(func, *args, **kwargs):
    """Attempts a function call up to MAX_RETRIES with exponential backoff."""
    for attempt in range(MAX_RETRIES):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            wait_time = 2**attempt  # Exponential backoff
            print(f"Error: {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
    print(f"Failed after {MAX_RETRIES} attempts.")
    return None

In [4]:
def get_team_info(seasons):
    """Fetches relevant team information for the specified seasons."""
    print("Fetching team information...")
    nba_teams = teams.get_teams()
    team_data = []

    for team in nba_teams:
        team_info = fetch_with_retries(
            TeamInfoCommon,
            team_id=team["id"],
            season_type_nullable="Regular Season",
            timeout=60,
        )
        if team_info:
            df_team = team_info.get_data_frames()[0]
            df_team = df_team[["TEAM_ID", "TEAM_ABBREVIATION"]]  # Only keep relevant features
            team_data.append(df_team)
            time.sleep(0.6)  # Delay to avoid API rate limits

    if team_data:
        df_teams = pd.concat(team_data, ignore_index=True)
        df_teams.to_csv("nba_team_data.csv", index=False)
    else:
        print("No team data fetched.")

# Run functions to save data to CSV files
get_team_info(seasons)
print("Team information data stored.")

Fetching team information...
Team information data stored.


In [5]:
def get_team_game_logs(seasons):
    """Fetches team game logs for the specified seasons and processes the MATCHUP column."""
    print("Fetching team game logs...")
    game_log_data = []

    for season in seasons:
        game_logs = fetch_with_retries(
            TeamGameLogs,
            season_nullable=season,
            season_type_nullable="Regular Season",
            timeout=60,
        )
        if game_logs:
            df_game_logs = game_logs.get_data_frames()[0]
            # Keep only relevant columns
            df_game_logs = df_game_logs[["GAME_ID", "GAME_DATE", "MATCHUP", "WL"]]
            game_log_data.append(df_game_logs)
            time.sleep(0.6)  # Delay to respect rate limits

    if game_log_data:
        # Concatenate all game logs
        df_all_game_logs = pd.concat(game_log_data, ignore_index=True)

        # Process MATCHUP column to create team1 and team2 columns
        matchups_split = df_all_game_logs['MATCHUP'].str.split(' @ | vs. ', expand=True)
        df_all_game_logs['TEAM1'] = matchups_split[0]
        df_all_game_logs['TEAM2'] = matchups_split[1]

        # Drop the original MATCHUP column if no longer needed
        df_all_game_logs.drop(columns=['MATCHUP'], inplace=True)

        # Extract and add SEASON_YEAR
        df_all_game_logs['SEASON_YEAR'] = pd.to_datetime(df_all_game_logs['GAME_DATE']).dt.year.astype(str)

        # Save the processed DataFrame to a CSV file
        df_all_game_logs.to_csv("nba_game_logs.csv", index=False)
        print("Processed game logs saved to 'nba_game_logs.csv'.")
    else:
        print("No game log data fetched.")

get_team_game_logs(seasons)
print("Team game logs data stored.")

Fetching team game logs...
Processed game logs saved to 'nba_game_logs.csv'.
Team game logs data stored.


In [6]:
def get_league_leaders():
    """Fetches league leaders data with relevant columns for analysis."""
    print("Fetching league leaders data...")
    leaders_data = fetch_with_retries(LeagueLeaders, timeout=60)
    if leaders_data:
        df_leaders = leaders_data.get_data_frames()[0]
        # Select only relevant columns
        relevant_columns = [
            "RANK", "PLAYER_ID", "PLAYER", "TEAM_ID", "TEAM", "GP", "MIN", "FGM", "FGA",
            "FG_PCT", "FG3M", "FG3A", "FG3_PCT", "FTM", "FTA", "FT_PCT", "OREB",
            "DREB", "REB", "AST", "STL", "BLK", "TOV", "PF", "PTS", "EFF"
        ]
        df_relevant_leaders = df_leaders[relevant_columns]
        df_relevant_leaders.to_csv("nba_league_leaders_relevant.csv", index=False)
    else:
        print("No league leaders data fetched.")

get_league_leaders()
print("League leaders data stored.")

Fetching league leaders data...
League leaders data stored.


In [7]:
import pandas as pd

# Load aggregated player data
player_aggregated_file = "nba_league_leaders_relevant.csv"  # Update with actual file path
team_abbreviation_file = "nba_team_data.csv"  # File containing TEAM_ID to TEAM_ABBREVIATION mapping

# Load player data and team abbreviation mapping
player_df = pd.read_csv(player_aggregated_file)
team_data_df = pd.read_csv(team_abbreviation_file)

# Merge team abbreviations into player data
player_df = player_df.merge(team_data_df, on="TEAM_ID", how="left")

# Convert numerical columns to float (force errors='coerce' to handle bad data)
player_df = player_df.apply(pd.to_numeric, errors='ignore')

# Ensure 'MIN' is numeric and replace any missing values with 1 to avoid division errors
player_df['MIN'] = pd.to_numeric(player_df['MIN'], errors='coerce').fillna(1)

# Define non-numerical columns to exclude
no_aggregate_columns = ['RANK', 'PLAYER_ID', 'PLAYER', 'TEAM_ID', 'TEAM', 'SEASON_YEAR', 'MIN']
numerical_columns = [col for col in player_df.columns if col not in no_aggregate_columns]

# Multiply each player's stats by their 'MIN' to weight the statistics
for col in numerical_columns:
    player_df[f"{col}_WEIGHTED"] = pd.to_numeric(player_df[col], errors='coerce').fillna(0) * player_df['MIN']

# Group by TEAM and sum the weighted stats & total minutes
grouped = player_df.groupby('TEAM')

team_aggregated_data = grouped[[f"{col}_WEIGHTED" for col in numerical_columns]].sum()

# Compute total minutes played by the team
team_aggregated_data['TOTAL_MIN'] = grouped['MIN'].sum()

# Normalize weighted stats by dividing by TOTAL_MIN (avoid division by zero)
for col in numerical_columns:
    team_aggregated_data[col] = team_aggregated_data[f"{col}_WEIGHTED"] / team_aggregated_data['TOTAL_MIN'].replace(0, 1)

# Drop weighted stat columns (no longer needed)
team_aggregated_data.drop(columns=[f"{col}_WEIGHTED" for col in numerical_columns], inplace=True)

# Add total games played per team (if column exists)
if 'GAMES_PLAYED' in player_df.columns:
    team_aggregated_data['TEAM_GAMES_PLAYED'] = grouped['GAMES_PLAYED'].sum()

# Reset index to flatten the DataFrame
team_aggregated_data.reset_index(inplace=True)

# Save the aggregated data
output_path = "nba_team_aggregated_data.csv"
team_aggregated_data.to_csv(output_path, index=False)

print(f"Team aggregated data saved to '{output_path}'.")


Team aggregated data saved to 'nba_team_aggregated_data.csv'.


<ipython-input-7-f9d5cb143435>:15: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  player_df = player_df.apply(pd.to_numeric, errors='ignore')


In [8]:
def extract_season_year(game_date):
    """
    Extracts the year from a game date in the format 'YYYY-MM-DDTHH:MM:SS'.

    Args:
        game_date (str): The game date string.

    Returns:
        str: The year as a string.
    """
    try:
        return pd.to_datetime(game_date).year
    except Exception as e:
        print(f"Error parsing date '{game_date}': {e}")
        return None

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.optimizers import Adam
import joblib
import tensorflow.keras.backend as K


def custom_accuracy(y_true, y_pred):
    """
    Custom accuracy metric to evaluate the model based on given conditions.
    """
    condition_1 = K.cast(y_pred < 0.5, dtype="float32") * K.cast(y_true == 0, dtype="float32")
    condition_2 = K.cast(y_pred >= 0.5, dtype="float32") * K.cast(y_true == 1, dtype="float32")
    return K.mean(condition_1 + condition_2)


def prepare_dataset(game_logs_file, features_file):
    """
    Prepare dataset for training using team-specific features.
    """
    try:
        # Load game logs and team features
        game_logs = pd.read_csv(game_logs_file, parse_dates=["GAME_DATE"])
        team_features = pd.read_csv(features_file)

        # Normalize key columns
        game_logs["TEAM1"] = game_logs["TEAM1"].str.strip().str.upper()
        game_logs["TEAM2"] = game_logs["TEAM2"].str.strip().str.upper()
        team_features["TEAM"] = team_features["TEAM"].str.strip().str.upper()

        # Merge features for TEAM1 and TEAM2
        game_logs = game_logs.merge(
            team_features.add_suffix("_TEAM1"),
            left_on="TEAM1",
            right_on="TEAM_TEAM1",
            how="left"
        ).merge(
            team_features.add_suffix("_TEAM2"),
            left_on="TEAM2",
            right_on="TEAM_TEAM2",
            how="left"
        )

        # Drop unnecessary columns
        columns_to_drop = ["TEAM_TEAM1", "TEAM_TEAM2", "GAME_DATE"]
        game_logs.drop(columns=[col for col in columns_to_drop if col in game_logs.columns], inplace=True)

        # Handle missing values
        game_logs.fillna(0, inplace=True)

        # Extract features and target
        feature_columns = game_logs.select_dtypes(include=np.number).columns.difference(["WL"])
        X = game_logs[feature_columns].to_numpy()
        y = (game_logs["WL"] == "W").astype(int).to_numpy()

        # Debugging and validation
        print("Features Shape:", X.shape)
        assert X.shape[0] == y.shape[0], "Mismatch between features and target sizes."
        assert not np.any(pd.isnull(X)), "Missing values found in features."

        return X, y, feature_columns

    except Exception as e:
        print("An error occurred in prepare_dataset:", e)
        raise


def build_neural_network(input_shape):
    """
    Build a neural network model.
    """
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(256, activation="relu"),
        Dropout(0.5),
        Dense(256, activation="relu"),
        Dropout(0.5),
        Dense(256, activation="relu"),
        Dropout(0.3),
        Dense(256, activation="relu"),
        Dropout(0.3),
        Dense(256, activation="relu"),
        Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss="binary_crossentropy",
                  metrics=[custom_accuracy])
    return model


def train_model(X, y, save_data_prefix="train_data"):
    """
    Train a neural network model and save the processed data.
    """
    # Normalize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Validate scaling
    assert not np.any(np.isnan(X_scaled)), "NaN values found after scaling."

    # Handle class imbalance
    X_resampled, y_resampled = SMOTE().fit_resample(X_scaled, y)

    # Save processed training data
    pd.DataFrame(X_resampled).to_csv(f"{save_data_prefix}_X.csv", index=False, header=False)
    pd.DataFrame(y_resampled).to_csv(f"{save_data_prefix}_y.csv", index=False, header=False)

    # Debugging output
    print(f"Processed training data saved to {save_data_prefix}_X.csv and {save_data_prefix}_y.csv.")
    print(f"Resampled Features Shape: {X_resampled.shape}, Resampled Target Shape: {y_resampled.shape}")

    # Build and train the model
    model = build_neural_network(X_resampled.shape[1])
    model.fit(X_resampled, y_resampled, epochs=1000, batch_size=10, validation_split=0.2)

    return model, scaler


def main():
    game_logs_file = "nba_game_logs.csv"
    features_file = "nba_team_aggregated_data.csv"

    # Prepare the dataset
    X, y, feature_columns = prepare_dataset(game_logs_file, features_file)

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Validate split
    assert X_train.shape[0] == y_train.shape[0], "Mismatch in training data sizes."
    assert X_test.shape[0] == y_test.shape[0], "Mismatch in test data sizes."

    # Train neural network
    model, scaler = train_model(X_train, y_train)

    # Save the model and scaler
    model.save("model.keras")
    joblib.dump(scaler, "scaler.pkl")

    # Evaluate the model
    X_test_scaled = scaler.transform(X_test)
    test_loss, test_custom_accuracy = model.evaluate(X_test_scaled, y_test)
    print(f"Test Loss: {test_loss}, Test Custom Accuracy: {test_custom_accuracy}")

    # Predictions
    y_pred = model.predict(X_test_scaled)
    pd.DataFrame({"Actual": y_test.flatten(), "Predicted": y_pred.flatten()}).to_csv("predictions.csv", index=False)
    print("Predictions saved to predictions.csv")


if __name__ == "__main__":
    main()


Features Shape: (1628, 46)
Processed training data saved to train_data_X.csv and train_data_y.csv.
Resampled Features Shape: (1320, 46), Resampled Target Shape: (1320,)
Epoch 1/1000
106/106 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - custom_accuracy: 0.4969 - loss: 0.6883 - val_custom_accuracy: 0.5381 - val_loss: 0.6275
Epoch 2/1000
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - custom_accuracy: 0.5027 - loss: 0.6697 - val_custom_accuracy: 0.5285 - val_loss: 0.6283
Epoch 3/1000
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - custom_accuracy: 0.5098 - loss: 0.6538 - val_custom_accuracy: 0.5489 - val_loss: 0.6192
Epoch 4/1000
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - custom_accuracy: 0.5151 - loss: 0.6312 - val_custom_accuracy: 0.5415 - val_loss: 0.6204
Epoch 5/1000
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - custom_accuracy: 0.5347 - loss: 0.6231 - val_custom_accuracy: 0.5259 - val_loss: 0.6188
Epoch 6/1000
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - custom_accuracy: 0.5354 - loss: 0.6187 - val_custom_acc

In [ ]:
import numpy as np
import pandas as pd
from nba_api.stats.static import teams
import joblib
from tensorflow.keras.models import load_model


def load_feature_names(feature_names_file="feature_names.pkl"):
    """
    Load saved feature names for feature alignment during prediction.
    """
    feature_names = joblib.load(feature_names_file)

    # Print feature names
    print("Feature Names Used in the Model:")
    print(feature_names)

    return feature_names


def get_team_id_by_abbreviation(team_abbreviation):
    """Retrieve the team ID by abbreviation."""
    nba_teams = teams.get_teams()
    for team in nba_teams:
        if team["abbreviation"].lower() == team_abbreviation.lower():
            return team["id"]
    raise ValueError(f"Team '{team_abbreviation}' not found! Please enter a valid abbreviation.")


def fetch_team_features(team_abbreviation, features_file, feature_names):
    """
    Retrieve the team-specific features for the given team abbreviation.

    Args:
        team_abbreviation: Abbreviation of the NBA team (e.g., 'LAL').
        features_file: CSV file containing the aggregated team features.
        feature_names: List of features expected by the model.

    Returns:
        numpy array of the team's features.
    """
    team_features = pd.read_csv(features_file)
    team_row = team_features[team_features["TEAM"] == team_abbreviation.upper()]

    if team_row.empty:
        raise ValueError(f"Features for team '{team_abbreviation}' not found in {features_file}.")

    # Ensure only numeric features are returned
    numeric_features = team_row[feature_names].apply(pd.to_numeric, errors='coerce')

    return numeric_features.to_numpy().flatten()


def predict_matchup_win_probability(team1_abbreviation, team2_abbreviation, features_file, model_path="model.h5", scaler_path="scaler.pkl"):
    """
    Predict the win probability for Team 1 in a matchup against Team 2.

    Args:
        team1_abbreviation: Abbreviation of Team 1 (e.g., 'LAL').
        team2_abbreviation: Abbreviation of Team 2 (e.g., 'BOS').
        features_file: CSV file containing aggregated team features.
        model_path: Path to the trained neural network model file.
        scaler_path: Path to the scaler file for feature normalization.
    """
    # Load model, scaler, and feature names
    model = load_model(model_path)
    scaler = joblib.load(scaler_path)
    feature_names = load_feature_names()

    # Fetch features for both teams
    team1_features = fetch_team_features(team1_abbreviation, features_file, feature_names)
    team2_features = fetch_team_features(team2_abbreviation, features_file, feature_names)

    # Create matchup feature differences and ratios
    matchup_features = np.concatenate([
        team1_features - team2_features,
        team1_features / (team2_features + 1e-5)  # Avoid division by zero
    ]).reshape(1, -1)

    # Scale the matchup features
    matchup_features_scaled = scaler.transform(matchup_features)

    # Predict win probability for Team 1
    win_probability = model.predict(matchup_features_scaled)[0][0]

    print(f"\nWin Probability for {team1_abbreviation} vs {team2_abbreviation}: {win_probability * 100:.2f}%")


def display_team_data():
    """
    Display available team abbreviations and names for user reference.
    """
    nba_teams = teams.get_teams()
    print("Available NBA Teams:")
    for team in nba_teams:
        print(f"{team['abbreviation']} - {team['full_name']}")


def main():
    """Main function to handle user input and prediction."""
    features_file = "features.csv"  # Path to the features file

    # Display team data before taking user input
    display_team_data()

    team1_abbreviation = input("Enter Team 1 abbreviation (e.g., 'LAL' for Los Angeles Lakers): ").strip()
    team2_abbreviation = input("Enter Team 2 abbreviation (e.g., 'BOS' for Boston Celtics): ").strip()

    try:
        predict_matchup_win_probability(team1_abbreviation, team2_abbreviation, features_file)
    except ValueError as e:
        print(f"Error: {e}")


if __name__ == "__main__":
    main()